In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from numpy.random import normal as normDist

In [2]:
dirCats=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
fileCat=r'06_COSMOS_SpectrZ_Zlim_1p2_AllBands_Errors.csv'

In [3]:
KB=pd.read_csv(os.path.join(dirCats,fileCat))
KB=KB.rename({'Seq_1':'Seq'},axis=1)
print(len(KB))
print(KB.columns)

19176
Index(['Seq', 'e_Ksmagap3', 'e_Ymagap3', 'e_Hmagap3', 'e_Jmagap3', 'e_Bmagap3',
       'e_Vmagap3', 'e_ipmagap3', 'e_rmagap3', 'e_umagap3',
       ...
       'r/IA679', 'r/IA738', 'r/NB711', 'ip/IA624', 'ip/IA679', 'ip/IA738',
       'ip/IA767', 'ip/NB711', 'ip/NB816', 'zpp/NB816'],
      dtype='object', length=173)


In [85]:
nNewGals=20
mags=list([col for col in KB.columns if ('ap3' in col and 'e_' not in col)])
mags.remove('zpmagap3')
errs=['e_'+mag for mag in mags]
idCol='Seq'

In [86]:
generateMags = lambda magVals, magErrs,nNewGals: normDist(magVals,magErrs,(1,nNewGals,len(magVals)))

In [87]:
def generateGals(galVector,mags,errs,idCol,nNewGals):
    data=generateMags(galVector[mags],galVector[errs],nNewGals)
    galsNew=pd.DataFrame(data=data[0],columns=mags)
    
    galsNew[idCol]=galVector[idCol]
    galsNew['z_spec']=galVector['z_spec']
    galsNew['zphot']=galVector['zphot']
    idValsNew=[str(galVector[idCol])+'_'+str(i) for i in range(nNewGals)]
    galsNew[idCol+'_new']=idValsNew
    return galsNew

In [88]:
data=KB[[idCol]+mags+errs+['z_spec','zphot']]

In [89]:
%%time
df=pd.DataFrame(columns=[idCol]+mags+['z_spec','zphot'])
i=0
for row in data.iterrows():
    ddf=generateGals(row[1],mags,errs,idCol,nNewGals)
    df=df.append(ddf,ignore_index=True)
    i+=1
    if i%200==0:
        print(i)

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000


KeyboardInterrupt: 

In [83]:
df.to_csv(os.path.join(dirCats,'06_COSMOS_SpectrZ_Zlim_1p2_AllBands_Augm.csv'),index=False)